In [ ]:
#insert your own twitter id via
import pandas as pd
import copy
from bs4 import BeautifulSoup
import requests as rq
import re
import tweepy
from AppCred import CONSUMER_KEY, CONSUMER_SECRET
from AppCred import ACCESS_TOKEN, ACCESS_TOKEN_SECRET
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

___

In [2]:

#handles that we need to extract from:
df_handles = pd.read_csv('list_mps.csv')
df_handles

,Unnamed: 0,screen_name
0,0,MaxovaRadka
1,1,KubiliusA
2,2,SuncanaGlavak
3,3,IECarvalhais
4,4,OKovarikMEP
...,...,...
527,527,jonasfernandez
528,528,KRLS
529,529,nicogoncas
530,530,paolodecastro


In [3]:
532/3

177.33333333333334

In [ ]:
#Selection of tweets that we each need to collect:

#Morten: 0-170
#Martin: 171-341
#Lezhong: 342-532

___

In [ ]:
#Set the iloc to the tweets you need
ex = df_handles['screen_name'].iloc[1:2] #REACHED 239 ()
ex

____

In [ ]:
#extract all the tweets from certain period, can seperate into several times


import tweepy
import datetime
import xlsxwriter
import sys
import pytz

# credentials from https://apps.twitter.com/


username = sys.argv[1]
startDate = datetime.datetime(2022, 2, 20, 0, 0, 0)
endDate =   datetime.datetime(2022, 5, 7, 0, 0, 0)

utc=pytz.UTC

startDate = utc.localize(startDate) 
endDate = utc.localize(endDate) 

tweets1 = []


for x in ex: #looping through ex that we made above

    tmpTweets = api.user_timeline(screen_name = x, tweet_mode = 'extended')
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets1.append(tweet)

    while (tmpTweets[-1].created_at > startDate):
        print("Last Tweet @", tmpTweets[-1].created_at, " - fetching some more")
        tmpTweets = api.user_timeline(screen_name = x, max_id = tmpTweets[-1].id, tweet_mode = 'extended')
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                tweets1.append(tweet)

    # second step- after we get the list for all of the tweets, we make them into a file.

    workbook = xlsxwriter.Workbook(x + ".xlsx")
    worksheet = workbook.add_worksheet()
    row = 1
    for tweet in tweets1:
        worksheet.write_string(row, 0, str(x))
        worksheet.write_string(row, 1, str(tweet.id))
        worksheet.write_string(row, 2, str(tweet.created_at))
        worksheet.write(row, 3, tweet.full_text)
        worksheet.write(row, 4, tweet.favorite_count)
        worksheet.write(row, 5, tweet.retweet_count)
        worksheet.write_string(row, 6, str(tweet.in_reply_to_status_id))
        row += 1

    workbook.close()
    print("Excel file ready")